In [ ]:
import pandas as pd
import numpy as np
from math import log
import sys
sys.path.insert(1, '../rtsvg/') # base location of the .py classes
from rtsvg import *
rt = RACETrack()

In [ ]:
#
# Source:  https://en.wikipedia.org/wiki/Apollo_13
# ... four sentence summary
# ... chatgpt gave me five though :(
#
_text = """Apollo 13 (April 11–17, 1970) was the seventh crewed mission in the Apollo space program and the third meant to land on the Moon. The craft was launched from Kennedy Space Center on April 11, 1970, but the lunar landing was aborted after an oxygen tank in the service module (SM) failed two days into the mission. The crew instead looped around the Moon in a circumlunar trajectory and returned safely to Earth on April 17. The mission was commanded by Jim Lovell, with Jack Swigert as command module (CM) pilot and Fred Haise as Lunar Module (LM) pilot. Swigert was a late replacement for Ken Mattingly, who was grounded after exposure to rubella.
A routine stir of an oxygen tank ignited damaged wire insulation inside it, causing an explosion that vented the contents of both of the SM's oxygen tanks to space. Without oxygen, needed for breathing and for generating electric power, the SM's propulsion and life support systems could not operate. The CM's systems had to be shut down to conserve its remaining resources for reentry, forcing the crew to transfer to the LM as a lifeboat. With the lunar landing canceled, mission controllers worked to bring the crew home alive.
Although the LM was designed to support two men on the lunar surface for two days, Mission Control in Houston improvised new procedures so it could support three men for four days. The crew experienced great hardship, caused by limited power, a chilly and wet cabin and a shortage of potable water. There was a critical need to adapt the CM's cartridges for the carbon dioxide scrubber system to work in the LM; the crew and mission controllers were successful in improvising a solution. The astronauts' peril briefly renewed public interest in the Apollo program; tens of millions watched the splashdown in the South Pacific Ocean on television.
An investigative review board found fault with preflight testing of the oxygen tank and Teflon being placed inside it. The board recommended changes, including minimizing the use of potentially combustible items inside the tank; this was done for Apollo 14. The story of Apollo 13 has been dramatized several times, most notably in the 1995 film Apollo 13 based on Lost Moon, the 1994 memoir co-authored by Lovell – and an episode of the 1998 miniseries From the Earth to the Moon."""
_summary_bard    = """Apollo 13 was a crewed mission to the Moon that was aborted after an oxygen tank failure.  The crew was forced to transfer to the Lunar Module as a lifeboat and improvise new procedures to survive.  The astronauts faced great hardship, including limited power, a chilly and wet cabin, and a shortage of potable water.  The crew successfully returned to Earth and the story was dramatized in the film Apollo 13."""
_summary_chatgpt = """Apollo 13 was the seventh crewed mission in the Apollo space program and was intended to land on the Moon. However, a failed oxygen tank in the service module forced the crew to abandon the lunar landing. They instead orbited around the Moon and returned safely to Earth on April 17, 1970. The mission faced numerous challenges, including limited resources, a damaged spacecraft, and the need to adapt the lunar module to support three astronauts instead of two. The story of Apollo 13 gained widespread attention and has been portrayed in various dramatizations, including the 1995 film Apollo 13."""

In [ ]:
model0, tokenizer0, device0 = rt.textCreateBertModel()
model,tokenizer,device = rt.__textTrainBertModel__(_text)

In [ ]:
def compareTwoSentences(i0_string, i1_string, model_x, tokenizer_x, device_x):
    mult0,    mult1    = 1.0, 1.0
    samples0, samples1 = 0,   0
    sum_log0, sum_log1 = 0,   0
    
    pred_tokens0, pred_probs0, as_tokens0, preds0, token_inputs0 = rt.__textBertWordProbabilities__(i0_string, model_x, tokenizer_x, device_x)
    pred_tokens1, pred_probs1, as_tokens1, preds1, token_inputs1 = rt.__textBertWordProbabilities__(i1_string, model_x, tokenizer_x, device_x)
    for i in range(max(len(pred_tokens0), len(pred_tokens1))):
        if i < len(pred_tokens0):
            as_input0,above0 = preds0[i][token_inputs0[i]],0
            for j in range(len(preds0[i])):
                if preds0[i][j] > as_input0:
                    above0 += 1
            mult0    *= pred_probs0[i]
            samples0 += 1
            sum_log0 += log(pred_probs0[i])

            pred_probs_holder0 = pred_probs0[i]
            as_tokens_holder0  = as_tokens0[i]
            above_holder0      = above0
        else:
            pred_probs_holder0 = 0
            as_tokens_holder0  = ''
            above_holder0      = 0

        if i < len(pred_tokens1):
            as_input1,above1 = preds1[i][token_inputs1[i]],0
            for j in range(len(preds1[i])):
                if preds1[i][j] > as_input1:
                    above1 += 1
            mult1    *= pred_probs1[i]
            samples1 += 1
            sum_log1 += log(pred_probs1[i])

            pred_probs_holder1 = pred_probs1[i]
            as_tokens_holder1  = as_tokens1[i]
            above_holder1      = above1
        else:
            pred_probs_holder1 = 0
            as_tokens_holder1  = ''
            above_holder1      = 0

        # print(f'{i:2} {pred_probs0[i]:0.2f} {as_tokens0[i]:12} {above0:4}\t\t||\t\t{pred_probs1[i]:0.2f} {as_tokens1[i]:12} {above1:4}')
        print(f'{i:2} {pred_probs_holder0:0.2f} {as_tokens_holder0:12} {above_holder0:4}\t\t||\t\t{pred_probs_holder1:0.2f} {as_tokens_holder1:12} {above_holder1:4}')

    print()
    print(f'sentence 0:   {mult0:0.2f}\t{mult0**(1.0/samples0):0.2f}\t{sum_log0/samples0:0.2f}')
    print(f'sentence 1:   {mult1:0.2f}\t{mult1**(1.0/samples1):0.2f}\t{sum_log1/samples1:0.2f}')

In [ ]:
i0_string = "Apollo 13 was the seventh crewed mission in the Apollo space program and the third meant to land on the Moon."
i1_string = "Mercury 12 was the seventh crewed mission in the Mercury space program and the third meant to land on the Sun."
print('**\n** Base Model\n**')
compareTwoSentences(i0_string, i1_string, model0, tokenizer0, device0)

In [ ]:
print('**\n** Trained Model\n**')
compareTwoSentences(i0_string, i1_string, model, tokenizer, device)

In [ ]:
i0_string = "Apollo 13 launched into space to go to the Moon."
i1_string = "Mercury 21 rocketed into space."
print('**\n** Base Model\n**')
compareTwoSentences(i0_string, i1_string, model0, tokenizer0, device0)

In [ ]:
print('**\n** Trained Model\n**')
compareTwoSentences(i0_string, i1_string, model, tokenizer, device)